In [56]:
import os
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
import re
import string

from tqdm import tqdm
# IMport language detection library
from langdetect import detect

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aravind/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/aravind/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Testing preprocess on single example

In [10]:
lyrics_sample_path = 'data/lyrics/ML1_There Goes My Baby.txt'
# Load the lyrics sample
with open(lyrics_sample_path, 'r') as f:
    lyrics_sample = f.read()
print(lyrics_sample)

13 ContributorsThere Goes My Baby Lyrics[Produced by Jim Jonsin and Rico Love]

[Intro]
Yeah... right...
Usher baby... okay, yeah man
Right... (turn the lights off)

[Chorus]
There goes my baby
(Oooh, girl, look at you)
You don't know how good it feels to call you my girl
There goes my baby
Loving everything you do
Oooh, girl, look at you

[Verse 1]
Bet you ain't know that I be checking you out when you be putting your heels on
I swear, your body's so perfect, baby
How you work it, baby yeah, wooo!
I love the way that you be poking it out, girl
Give me something to feel on
So please believe we gone be twerking it out
By the end of the night, baby

[Pre-Chorus]
I've been waiting all day to wrap my hands
Around your waist and kiss your face
Wouldn't trade this feeling for nothing
(Not even for a minute)
And I'll sit here long as it takes
To get you all alone
But as soon as you come walking my way
You gon' hear me say
See Usher LiveGet tickets as low as $56You might also like[Chorus]
Ther

In [11]:
# Lyric sample location
lyrics = lyrics_sample[lyrics_sample.find('\n')+1:]
lyrics = lyrics.split('Embed', 1)[0]
lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
# Remove everything inside () and if there is a () occupying the whole line, remove the whole line
lyrics = re.sub(r'\([^)]*\)|^\([^)]*\)$', '', lyrics, flags=re.MULTILINE)
# Remove the numbers
lyrics = re.sub(r'\d+', '', lyrics)
# Remove the punctuations except for ' onces between two characters
lyrics = re.sub(r'[^\w\s\']', '', lyrics)

# Remove any \n in the beginning
lyrics = lyrics.lstrip('\n')
# Split the lyrics into lines
lines = lyrics.split('\n')
# Remove stop words but ignore '\n'
stop_words = set(stopwords.words('english'))
# Process each line separately
processed_lines = []
for line in lines:
    # Process parentheses removal and check if line is not empty
    processed_line = re.sub(r'\([^)]*\)', '', line)
    if processed_line.strip():
        words = [word for word in processed_line.split() if word not in stop_words]
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        processed_lines.append(' '.join(words))
# Join the processed lines into a single string
processed_lyrics = '\n'.join(processed_lines)
# Convert to lowercase
processed_lyrics = processed_lyrics.lower()
print(processed_lyrics)



yeah right
usher baby okay yeah man
right
there go baby
you know good feel call girl
there go baby
loving everything
oooh girl look
bet ain't know i checking putting heel
i swear body's perfect baby
how work baby yeah wooo
i love way poking girl
give something feel
so please believe gone twerking
by end night baby
i've waiting day wrap hand
around waist kiss face
wouldn't trade feeling nothing
and i'll sit long take
to get alone
but soon come walking way
you gon' hear say
see usher liveget ticket low you might also like
there go baby
you know good feel call girl
there go baby
loving everything
oooh girl look
i get chill whenever i see face place girl
feel like i'm movie baby
i'm like oowee baby
like waterfall hair fall waist
can i get taste girl
no need creep 'cause baby i ain't ashamed
of calling name girl
i've waiting day wrap hand
around waist kiss face
wouldn't trade feeling nothing
not even minute
and i'll sit long take
to get alone
but soon come walking way
you gon' hear say
ther

# Iterate for entire dataset

In [57]:
# Get the list of files in the lyrics folder
lyrics_files = os.listdir('data/lyrics/')
# Get the first files in .txt format starting with ML
lyrics_files = [f for f in lyrics_files if f.startswith('ML') and f.endswith('.txt')]
# Print the list of files
print(lyrics_files)

['ML1569_Songbird.txt', 'ML1303_Nervous Breakdown.txt', 'ML1433_Never My Love.txt', 'ML572_Its For You.txt', 'ML1445_Say It.txt', 'ML1835_Believe.txt', 'ML1453_Lets Fight.txt', 'ML309_Kiss On My List.txt', 'ML1172_The Air That I Breathe.txt', 'ML229_Fantasy Girl .txt', 'ML1216_All I Want Is You.txt', 'ML1991_Liar.txt', 'ML223_Rusty Cage.txt', 'ML1169_Listen People.txt', 'ML183_My Name Is Victory .txt', 'ML1913_Lullaby Of Broadway.txt', 'ML169_Burn.txt', 'ML737_The Metal.txt', 'ML1299_Napalm Love.txt', 'ML83_Somebody To Love.txt', 'ML768_Hit Me With Your Rhythm Stick.txt', 'ML1503_The Glory Of Love.txt', 'ML574_The Velvet Cell.txt', 'ML824_Aganju.txt', 'ML699_Bad Time.txt', 'ML1859_Legs Of Bees.txt', 'ML182_God Has Not 4got.txt', 'ML1365_You Dont Have To Be Afraid.txt', 'ML97_You Got That Right.txt', 'ML1809_Angry World.txt', 'ML155_Im Too Sexy.txt', 'ML333_Glory To God Forever.txt', 'ML498_Remind Me LP Version.txt', 'ML819_There Is No Greater Love.txt', 'ML75_Your Man.txt', 'ML846_El L

In [58]:
# Print size of the list
print(len(lyrics_files))

1980


In [59]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [60]:
# Iterate over the lyrics files and load it and detect language
max_list = []
lyrics_df = pd.DataFrame(columns=['song_id', 'lyrics'])
for i in tqdm(lyrics_files):
    # Lyric sample location
    lyrics_sample_path = 'data/lyrics/'+i
    # Load the lyrics sample
    with open(lyrics_sample_path, 'r') as f:
        lyrics_sample = f.read()
    lyrics = lyrics_sample[lyrics_sample.find('\n')+1:]
    lyrics = lyrics.split('Embed', 1)[0]
    lyrics = re.sub(r'\[[^\]]*\]', '', lyrics)
    # Remove everything inside () and if there is a () occupying the whole line, remove the whole line
    lyrics = re.sub(r'\([^)]*\)|^\([^)]*\)$', '', lyrics, flags=re.MULTILINE)
    # Remove the numbers
    lyrics = re.sub(r'\d+', '', lyrics)
    # Remove the punctuations except for ' onces between two characters
    lyrics = re.sub(r'[^\w\s\']', '', lyrics)
    # Remove any 'You might also like' and everything after it in that line
    lyrics = re.sub(r'You might also like.*', '', lyrics)
    # Remove any \n in the beginning
    lyrics = lyrics.lstrip('\n')
    # Split the lyrics into lines
    lines = lyrics.split('\n')
    # Remove stop words but ignore '\n'
    stop_words = set(stopwords.words('english'))
    # Process each line separately
    processed_lines = []
    for line in lines:
        # Process parentheses removal and check if line is not empty
        processed_line = re.sub(r'\([^)]*\)', '', line)
        if processed_line.strip():
            words = [word for word in processed_line.split() if word not in stop_words]
            lemmatizer = WordNetLemmatizer()
            words = [lemmatizer.lemmatize(word) for word in words]
            processed_lines.append(' '.join(words))
    # Join the processed lines into a single string
    processed_lyrics = '\n'.join(processed_lines)
    # Convert to lowercase
    processed_lyrics = processed_lyrics.lower()
    # Detect language
    language = detect(processed_lyrics)
    song_id = i.split('_')[0]
    # If the language is english, add it to the df
    if language == 'en':
        tokenizer.pad_token = tokenizer.eos_token
        tokenized_lyrics = tokenizer(processed_lyrics, return_tensors='pt',  truncation=True, padding=True)
        if len(tokenized_lyrics.input_ids[0]) < 1024:
            max_list.append([i,song_id, len(tokenized_lyrics.input_ids[0])])
            # Add to lyrics_df
            lyrics_df.loc[len(lyrics_df)] = [song_id, processed_lyrics]
    # else:
    #     print('The language of the song {} is not English'.format(i))

100%|██████████| 1980/1980 [00:13<00:00, 142.88it/s]


In [61]:
lyrics_df

,song_id,lyrics
0,ML1569,for there'll cry\nfor sun shining\nand i feel ...
1,ML1303,a mental shakedown\nsee hand shiver\nsee knee ...
2,ML1433,you ask there'll come time\nwhen i grow tired\...
3,ML572,she's something\nand i can't breathe\nam\nand ...
4,ML1445,never stop saying mine\ntell ever ever sweet\n...
...,...,...
1716,ML1837,no one seems around\ntoday must've gone withou...
1717,ML508,hello friend\ni remember\nso alive wide eye 't...
1718,ML1641,entombs lifeless\nthe darkest black cast\nthe ...
1719,ML1079,the little ordinary thing everyone ought\nim l...


In [62]:
df_mood = pd.read_excel('data/MoodyLyrics/ml_balanced.xlsx')
df_mood.head(10)

,Index,Artist,Title,Mood
0,ML1,Usher,There Goes My Baby,relaxed
1,ML2,Da'Ville,On My Mind,relaxed
2,ML3,Rihanna,Rockstar 101,relaxed
3,ML4,J. Holiday,Bed,relaxed
4,ML5,Morgan Heritage,Don't Haffi Dread,angry
5,ML6,Whitney Houston,My Love Is Your Love,happy
6,ML7,Serani,Study People,angry
7,ML8,Austra,Spellwork,angry
8,ML9,Coro,Can't Let You Go,happy
9,ML10,Beenie Man,Slam,relaxed


In [63]:
# Mapping
lyrics_df['Mood'] = ''
for i in range(len(lyrics_df)):
    lyrics_df['Mood'][i] = df_mood['Mood'][i]
lyrics_df.head()

,song_id,lyrics,Mood
0,ML1569,for there'll cry\nfor sun shining\nand i feel ...,relaxed
1,ML1303,a mental shakedown\nsee hand shiver\nsee knee ...,relaxed
2,ML1433,you ask there'll come time\nwhen i grow tired\...,relaxed
3,ML572,she's something\nand i can't breathe\nam\nand ...,relaxed
4,ML1445,never stop saying mine\ntell ever ever sweet\n...,angry


In [67]:
lyrics_df['Mood'].value_counts()

Mood
happy      500
relaxed    423
sad        415
angry      383
Name: count, dtype: int64

In [68]:
lyrics_df['Mood'].value_counts()
# Do a encoding to assign numbers to the mood   
lyrics_df['Mood_encod'] = lyrics_df['Mood'].astype('category')
lyrics_df['Mood_encod'] = lyrics_df['Mood_encod'].cat.codes
# Print the codes
print(lyrics_df['Mood_encod'].value_counts())

Mood_encod
1    500
2    423
3    415
0    383
Name: count, dtype: int64


In [69]:
# Happy = 1, Relax = 2, Sad = 3, Angry = 0

In [75]:
# Sort by song_id = song_id..split('_')[0]
lyrics_df = lyrics_df.sort_values(by=['song_id'])
lyrics_df.head(5)

,song_id,lyrics,Mood,Mood_encod
1309,ML1,yeah right\nusher baby okay yeah man\nright\nt...,relaxed,2
1014,ML10,well like menace society\nhahaha\nbiggin' fi d...,happy,1
128,ML100,one two one two three four\nwell i take whatev...,sad,3
385,ML1000,well someone told yesterday\nthat throw love a...,angry,0
158,ML1001,cheeks flushed apple red\nringing every day\nb...,angry,0


In [78]:
# Shuffle and print the df
lyrics_df.sample(frac=1).reset_index(drop=True).head(5)

,song_id,lyrics,Mood,Mood_encod
0,ML1849,youve got chain\ntied tight tie\nwhos got claw...,happy,1
1,ML1791,standing disarray\na calm serenity\nthe wind c...,happy,1
2,ML890,you bring joy\nwhen i'm\noh much joy\nwhen i l...,sad,3
3,ML960,howlin' way i'm comin' home\non hand knee i'm ...,sad,3
4,ML375,a warning people\nthe good evil\nthis war\nto ...,sad,3


In [72]:
# Print ML79's lyrics, mood
print(lyrics_df[lyrics_df['song_id'] == 'ML79']['lyrics'].values[0])

lie tell everything right
lie tell you'r stayin' night
tell never leave
and i'll try make believe
that everything tell true
come baby
lie go ahead lie
lie go ahead lie
lie matter anymore
it could never way
if i can't hold
leave somethin' i hold onto
for little let dream
oh anyone see
that love
but right baby let pretend
that love never end
lie go ahead lie
lie go ahead lie
lie go ahead lie
lie go ahead lie


In [73]:
# Save the lyrics_df as a csv file'
lyrics_df.to_csv('data/lyrics_preprocessed_final.csv', index=False)